In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [4]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4.1-mini'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [5]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [6]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

In [7]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [8]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [9]:
from crewai_tools import BaseTool

In [10]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        # Try any action
        return "positive"

In [11]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [12]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [13]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [14]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

In [15]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: To create a comprehensive profile on DeepLearningAI, I need to gather detailed information on the company background, key personnel, recent milestones, and potential needs. Since I have no direct files related to DeepLearningAI yet, I will start by searching the internet for relevant data about DeepLearningAI.

Action: Search the internet  
Action Input: {"search_query": "DeepLearningAI company b

In [16]:
from IPython.display import Markdown
Markdown(result)

---

**Email Draft 1: Congratulating on the Recent Launch & Introducing CrewAI’s Strategic Support**

Subject: Congratulations on Your Recent Launch – Let’s Elevate DeepLearningAI’s Impact Together

Dear Dr. Ng,

I wanted to personally congratulate you and the entire DeepLearningAI team on your recent product launch. It’s inspiring to see how your vision continues to transform AI education and empower learners worldwide.

At CrewAI, we specialize in scalable, AI-driven solutions that seamlessly integrate with existing platforms to enhance personalized learning experiences. We believe our technology can help DeepLearningAI not only meet the evolving demands of your growing user base but also pioneer new frontiers in adaptive learning.

I would love to explore how we can support your mission to democratize AI education through strategic collaboration and innovative technology.

Looking forward to the opportunity to discuss this further.

Warm regards,  
[Your Name]  
Lead Sales Representative, CrewAI  

---

**Email Draft 2: Highlighting Scalability and Innovation Partnership**

Subject: Scaling DeepLearningAI’s Vision with AI-Driven Personalization and Innovation

Hi Andrew,

DeepLearningAI’s commitment to accessible, cutting-edge AI education is truly remarkable. With your recent advancements, the potential to reach even more learners is immense.

CrewAI offers scalable solutions designed to grow alongside your platform, ensuring every learner benefits from a tailored, engaging experience. Beyond scalability, we’re excited about the prospect of co-developing advanced learning analytics and adaptive technologies that can further enhance DeepLearningAI’s offerings.

I’d welcome the chance to discuss how CrewAI can be a trusted innovation partner on your journey.

Best regards,  
[Your Name]  
Lead Sales Representative, CrewAI  

---

**Email Draft 3: Emphasizing Long-Term Collaboration and Seamless Integration**

Subject: Partnering for Long-Term Success at DeepLearningAI

Dear Andrew,

DeepLearningAI’s leadership in AI education is setting a new standard for the industry. As you continue to scale and innovate, CrewAI’s solutions can seamlessly integrate with your current systems, driving operational efficiency and deeper learner engagement.

We view this as the start of a strategic partnership focused on long-term growth and shared innovation, aligning perfectly with your mission to accelerate AI learning worldwide.

I look forward to connecting and exploring how we can support your exciting journey ahead.

Kind regards,  
[Your Name]  
Lead Sales Representative, CrewAI  

---

These drafts are crafted to resonate with DeepLearningAI’s values of innovation, education, and scalability, while positioning CrewAI as a collaborative partner aligned with their goals and recent achievements.